In [1]:
import os
# set the current working directory to the deployed package folder. This is required by isaac.
# This cell should only run once.
os.chdir("../../../")
os.getcwd()

'/root/.cache/bazel/_bazel_root/4ba0b84eadbbe44ac95feee54d73b56c/execroot/com_nvidia_isaac_sdk/bazel-out/k8-opt/bin/packages/universal_robots/apps/simple_joint_control.runfiles/com_nvidia_isaac_sdk'

In [2]:
from IPython.display import display
import json
import numpy as np
import time

from packages.pyalice import Application, Codelet, Message
from packages.pyalice.gui.composite_widget import CompositeWidget

np.set_printoptions(precision=3)

In [3]:
# A Python codelet for joint control through widget
class JointPositionControl(Codelet):
    def start(self):
        self.rx = self.isaac_proto_rx("CompositeProto", "state")
        self.tx = self.isaac_proto_tx("CompositeProto", "command")
        
        joints = self.config.joints
        limits = self.config.limits
        measure = self.config.measure
        self.stop_control = 3
        self._widget = CompositeWidget(joints, measure, limits)
        if self._widget is None:
            report_failure("Cannot create valid widget")
            return
        display(self._widget.panel)

        self.tick_periodically(0.1)

    def tick(self):
        state_msg = self.rx.message
        if state_msg is None:
            return
        self._widget.composite = state_msg

        self.tx._msg = self._widget.composite
        if self.tx._msg is not None:
            self.tx.publish()
            
# set kinematic file and get list of joints
kinematic_file = "apps/assets/kinematic_trees/ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

# Obtain the UR10 IP from UR Console and update this
UR10_IP = "127.0.0.1"

['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']


UR10 Hardware Joint Control
======

In [5]:
app = Application(name="simple_joint_control_ur10_hardware")

# UR driver subgraph, update according to e series or cb3 robot.
app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_eseries_robot.subgraph.json", prefix="ur")
#app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_cb3_robot.subgraph.json", prefix="ur")

ur_interface = app.nodes["ur.subgraph"]["interface"]

# configs
ur_controller = app.nodes["ur.controller"]["ScaledMultiJointController"]
ur_controller.config.control_mode = "joint speed"

ur_driver = app.nodes["ur.universal_robots"]["UniversalRobots"]
ur_driver.config.control_mode = "joint speed"
ur_driver.config.robot_ip = UR10_IP
ur_driver.config.headless_mode = False

ur_dc = app.nodes["ur.dashboard_client_isaac"]["DashboardClientIsaac"]
ur_dc.config.robot_ip = UR10_IP

# add pycodelet for joint position control
widget_node = app.add("command_generator")
joint_commander = widget_node.add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = 'position'

# edges
app.connect(joint_commander, "command", ur_interface, "joint_target")
app.connect(ur_interface, "arm_state", joint_commander, "state")

# Visualize channels in sight, the interface can be found on 127.0.0.1:3000
# sight_plot = app.add("sight_widgets").add(app.registry.isaac.sight.SightWidget, "plot")

# start application
app.start()


2021-03-18 14:16:12,303 DEBUG Binding PyCodelet command_generator/PyCodelet
2021-03-18 14:16:12,307 DEBUG Launching isaac core
2021-03-18 14:16:12,309 DEBUG Launching pycodelet threads
2021-03-18 14:16:12,310 DEBUG Launching command_generator/PyCodelet


In [6]:
app.stop()

2021-03-17 14:28:00,648 DEBUG Stopped command_generator/PyCodelet
2021-03-17 14:28:00,649 DEBUG Python Codelets All stopped...


UR10 Digital IO Control
=====

In [4]:
app = Application(name="io_control_ur10_hardware")

# UR driver subgraph, update according to e series or cb3 robot.
app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_eseries_robot.subgraph.json", prefix="ur")
#app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_cb3_robot.subgraph.json", prefix="ur")

ur_interface = app.nodes["ur.subgraph"]["interface"]


ur_driver = app.nodes["ur.universal_robots"]["UniversalRobots"]
io_names = ur_driver.config.tool_digital_out_names
ur_driver.config.robot_ip = UR10_IP

# add pycodelet for digitalIO control
widget_node = app.add("command_generator")
io_commander = widget_node.add(JointPositionControl)
io_commander.config.joints = io_names
io_commander.config.limits = [[0, 1]] * len(io_names)
io_commander.config.measure = 'none'
app.connect(io_commander, "command", ur_interface, "io_command")
app.connect(ur_interface, "io_state", io_commander, "state")

app.start()

2021-02-17 12:42:13,898 DEBUG Binding PyCodelet command_generator/PyCodelet
2021-02-17 12:42:13,900 DEBUG Launching isaac core
2021-02-17 12:42:13,903 DEBUG Launching pycodelet threads
2021-02-17 12:42:13,905 DEBUG Launching command_generator/PyCodelet


In [5]:
app.stop()

2021-02-17 12:43:12,935 DEBUG Stopped command_generator/PyCodelet
2021-02-17 12:43:12,935 DEBUG Python Codelets All stopped...
